在此之前，已经从10x Genome上下载得到了原始数据。

Quality Control

In [20]:
import scanpy as sc
import pandas as pd
import scipy.sparse as sp
from scipy import io
from sklearn.feature_extraction.text import TfidfTransformer
from intervaltree import IntervalTree
from sklearn.decomposition import TruncatedSVD
import os
outdir = r"D:\HuaweiMoveData\Users\hya\Desktop\CMML\ICA2\Results"
os.makedirs(outdir, exist_ok=True)

# 载入数据
frag_file = r"D:\HuaweiMoveData\Users\hya\Desktop\CMML\ICA2\Downloaded_Raw_Data\pbmc_unsorted_3k_atac_fragments.tsv"
peaks_bed = r"D:\HuaweiMoveData\Users\hya\Desktop\CMML\ICA2\Downloaded_Raw_Data\pbmc_unsorted_3k_atac_peaks.bed"

# 先把 peaks.bed 读进来，按染色体构建 IntervalTree
peak_list = [] # 保存所有peak的(chr, start, end)信息
peak_trees = {} # 染色体名称chr为键，IntervalTree对象为值

with open(peaks_bed) as f:
    for peak_id, line in enumerate(f):
        chrom, s, e = line.strip().split()[:3]
        s, e = int(s), int(e)
        peak_list.append((chrom, s, e))
        tree = peak_trees.setdefault(chrom, IntervalTree())
        tree[s:e] = peak_id

n_peaks = len(peak_list)

# 逐行扫描 fragments.tsv，查区间树，将每条 fragment 分配给所有重叠的 peak
cell_to_id = {} # barcode -> col index
rows, cols, data = [], [], []

with open(frag_file) as f:
    for line in f:
        chrom, s, e, barcode, cnt = line.strip().split()[:5]
        s, e, cnt = int(s), int(e), int(cnt)
        if chrom not in peak_trees:
            continue
        for iv in peak_trees[chrom].overlap(s, e):
            peak_id = iv.data
            # barcode → cell_id
            if barcode not in cell_to_id:
                cell_to_id[barcode] = len(cell_to_id)
            cell_id = cell_to_id[barcode]
            rows.append(peak_id)
            cols.append(cell_id)
            data.append(cnt)

n_cells = len(cell_to_id)

# 构造 CSR 稀疏矩阵（会自动把重复的 (peak,cell) 累加）
mat = sp.coo_matrix((data, (rows, cols)), shape=(n_peaks, n_cells)).tocsr()

# AnnData 封装 (cells × peaks)
adata_ATAC = sc.AnnData(mat.T.copy())
adata_ATAC.var_names = [f"{c}:{s}-{e}" for c, s, e in peak_list]
adata_ATAC.obs_names = list(cell_to_id.keys())

# QC: 过滤低质量细胞
adata_ATAC.obs['n_fragments'] = adata_ATAC.X.sum(axis=1).A1
adata_ATAC = adata_ATAC[adata_ATAC.obs['n_fragments'] >= 1000, :]

LSI: TF-IDF Normalization

In [21]:
# TF–IDF 归一化
tfidf = TfidfTransformer(norm='l1', use_idf=True, smooth_idf=True)
X_tfidf = tfidf.fit_transform(adata_ATAC.X)
adata_ATAC.obsm['X_tfidf'] = X_tfidf # 存回 AnnData.obsm 以便后续使用

C:\Users\hya\AppData\Local\Temp\ipykernel_29676\1781549544.py:4: ImplicitModificationWarning: Setting element `.obsm['X_tfidf']` of view, initializing view as actual.
  adata_ATAC.obsm['X_tfidf'] = X_tfidf # 存回 AnnData.obsm 以便后续使用


Alignment & 导出结果文件，用于准备scAI的输入文件

In [23]:
# 对齐两端,只保留交集
adata_RNA = sc.read_h5ad(r"D:\HuaweiMoveData\Users\hya\Desktop\CMML\ICA2\Results\RNA_preprocessed.h5ad")
common = adata_RNA.obs_names.intersection(adata_ATAC.obs_names)

adata_RNA  = adata_RNA[common, :].copy()
adata_ATAC = adata_ATAC[common, :].copy()

print("对齐后细胞数：", adata_RNA.n_obs, adata_ATAC.n_obs)
# 现在两者都应该等于 1081

# ATAC
# 保存完整的 AnnData (包含 X、obs、var、obsm 等所有信息)
adata_ATAC.write(
    r"D:\HuaweiMoveData\Users\hya\Desktop\CMML\ICA2\Results\ATAC_preprocessed.h5ad"
)

# 保存 counts 矩阵 (peaks × cells)，peak，barcode 标签
tfidf_mat = adata_ATAC.obsm['X_tfidf'].copy().T
io.mmwrite(
    r"D:\HuaweiMoveData\Users\hya\Desktop\CMML\ICA2\Results\scAI_atac_counts.mtx",
    sp.csr_matrix(tfidf_mat)
)
pd.Series(adata_ATAC.var_names, name='peak').to_csv(
    r"D:\HuaweiMoveData\Users\hya\Desktop\CMML\ICA2\Results\scAI_atac_peaks.tsv",
    sep="\t",
    index=False
)
pd.Series(adata_ATAC.obs_names, name='cell').to_csv(
    r"D:\HuaweiMoveData\Users\hya\Desktop\CMML\ICA2\Results\scAI_atac_barcodes.tsv",
    sep="\t",
    index=False
)

# RNA
# 保存完整的 AnnData (包含 X、obs、var、obsm 等所有信息)
adata_RNA.write(
    r"D:\HuaweiMoveData\Users\hya\Desktop\CMML\ICA2\Results\RNA_preprocessed.h5ad"
)

# 保存counts矩阵(genes × cells)，gene，barcode标签
rna_counts = adata_RNA.X.copy().T
io.mmwrite(
    r"D:\HuaweiMoveData\Users\hya\Desktop\CMML\ICA2\Results\scAI_rna_counts.mtx",
    sp.csr_matrix(rna_counts)
)
pd.Series(adata_RNA.var_names, name='gene').to_csv(
    r"D:\HuaweiMoveData\Users\hya\Desktop\CMML\ICA2\Results\scAI_rna_genes.tsv",
    sep="\t",
    index=False
)
pd.Series(adata_RNA.obs_names, name='cell').to_csv(
    r"D:\HuaweiMoveData\Users\hya\Desktop\CMML\ICA2\Results\scAI_rna_barcodes.tsv",
    sep="\t",
    index=False
)

对齐后细胞数： 1081 1081


LSI: Singular Value Decomposition

In [ ]:
# 6) LSI (SVD 降维)
svd = TruncatedSVD(n_components=50, random_state=0)
adata_atac.obsm['X_lsi'] = svd.fit_transform(X_tfidf)